In [60]:
import os

FOLDER_ID = os.getenviron("FOLDER")
SECRET_TOKEN = ""
SERVICE_URL = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
OAUTH_TOKEN = os.getenviron("OAUTH")
IAM_TOKEN = ""

In [1]:
!curl -X POST \
  -d '{"yandexPassportOauthToken":"YOUR TOKEN HERE"}' \
  https://iam.api.cloud.yandex.net/iam/v1/tokens

{
 "code": 16,
 "message": "OAuth token is invalid or expired",
 "details": [
  {
   "@type": "type.googleapis.com/google.rpc.RequestInfo",
   "requestId": "4970d6a1-78df-40f9-9e7b-5b1d5b56f7b4"
  }
 ]
}


In [1]:
import pandas as pd

df = pd.read_parquet("hf://datasets/mteb/summeval/data/test-00000-of-00001-35901af5f6649399.parquet")


In [2]:
df.head()

,machine_summaries,human_summaries,relevance,coherence,fluency,consistency,text,id
0,"[donald sterling , nba team last year . sterli...",[V. Stiviano must pay back $2.6 million in gif...,"[1.6666666666666667, 1.6666666666666667, 2.333...","[1.3333333333333333, 3.0, 1.0, 2.6666666666666...","[1.0, 4.666666666666667, 4.333333333333333, 4....","[1.0, 2.3333333333333335, 4.666666666666667, 5...",(CNN)Donald Sterling's racist remarks cost him...,cnn-test-404f859482d47c127868964a9a39d1a7645dd2e9
1,[north pacific gray whale has earned a spot in...,"[The whale, Varvara, swam a round trip from Ru...","[2.3333333333333335, 4.666666666666667, 3.6666...","[1.3333333333333333, 4.666666666666667, 3.6666...","[1.0, 5.0, 4.666666666666667, 3.66666666666666...","[1.3333333333333333, 5.0, 5.0, 4.3333333333333...",(CNN)A North Pacific gray whale has earned a s...,cnn-test-4761dc6d8bdf56b9ada97104113dd1bcf4aed3f1
2,[russian fighter jet intercepted a u.s. reconn...,[The incident occurred on April 7 north of Pol...,"[4.0, 4.0, 4.0, 3.3333333333333335, 3.33333333...","[3.3333333333333335, 4.333333333333333, 1.6666...","[3.6666666666666665, 4.333333333333333, 5.0, 4...","[5.0, 5.0, 4.666666666666667, 5.0, 5.0, 5.0, 5...",(CNN)After a Russian fighter jet intercepted a...,cnn-test-5139ccfabee55ddb83e7937f5802c0a67aee8975
3,[michael barnett captured the fire on intersta...,[Country band Lady Antebellum's bus caught fir...,"[2.0, 3.0, 2.6666666666666665, 3.3333333333333...","[2.0, 3.0, 2.6666666666666665, 3.3333333333333...","[2.6666666666666665, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[2.3333333333333335, 5.0, 5.0, 5.0, 5.0, 5.0, ...",(CNN)Lady Antebellum singer Hillary Scott's to...,cnn-test-88c2481234e763c9bbc68d0ab1be1d2375c1349a
4,[deep reddish color caught seattle native tim ...,[Smoke from massive fires in Siberia created f...,"[1.6666666666666667, 3.6666666666666665, 3.333...","[1.6666666666666667, 3.6666666666666665, 1.666...","[5.0, 5.0, 5.0, 5.0, 4.666666666666667, 5.0, 5...","[2.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",(CNN)A fiery sunset greeted people in Washingt...,cnn-test-a02e362c5b8f049848ce718b37b96117485461cf


In [63]:
import copy
import json
import requests
import re
import string
import time

import pandas as pd

from tqdm import tqdm

body_template = {
    "sourceLanguageCode": "en",
    "targetLanguageCode": "ru",
    "format": "PLAIN_TEXT",
    "texts": [],
    "folderId": "b1gsu7em2r0of7ie35bd",
    "speller": True
}
headers = {"Authorization": f"Bearer {IAM_TOKEN}", "Content-Type": "application/json", "x-folder-id": f"{FOLDER_ID}"}
pattern = re.compile(r'[\W_]+')

def send_yatranslate_request(texts):
    body = copy.deepcopy(body_template)

    body["texts"] = texts
    while True:
        try:
            r = requests.post(SERVICE_URL, json=body, headers=headers)
            r.raise_for_status()
            break
        except Exception as e:
            print(f"Caught exception {e}")
            time.sleep(10)
    time.sleep(3)
    return r

In [55]:
from tqdm import tqdm

translated_texts = []
for row in tqdm(df.iterrows()):
    request_data = list(row[1]["machine_summaries"])
    r = send_yatranslate_request(request_data)

    for txt in r.json()["translations"]:
        translated_texts.append(txt["text"])


100it [08:52,  5.33s/it]


In [50]:
print(translated_texts[-5:-1)

['дональд Стерлинг, игрок прошлогодней команды НБА . жена Стерлинга подала в суд на подарки на сумму 2,6 миллиона долларов. стерлинг говорит, что он - бывшая компаньонка, которая проиграла конкурс. стерлинг обязал В. Стивиано выплатить 2,6 миллиона долларов в виде подарков после того, как его жена подала в суд. стерлинг также приобрел костюм пасхального кролика стоимостью 391 доллар, 299 долларов и еще 299 долларов.', 'Дональд Стерлинг обвинил Стивиано в том, что он нацелился на очень богатых пожилых мужчин. она утверждала, что Дональд Стерлинг использовал деньги пары, чтобы купить Стивиано ferrari, два bentley и range rover. Стивиано возразил, что нет ничего плохого в том, что Дональд Стерлинг дарит ей подарки.', 'судья Лос-Анджелеса обязал В. Стивиано выплатить более 2,6 миллионов долларов в виде подарков после того, как жена Стерлинга подала на нее в суд. -lrb- cnn -rrb- расистские высказывания Дональда Стерлинга в прошлом году стоили ему команды НБА. но теперь его бывшая спутница ж

In [44]:
print(r.json())

{'translations': [{'text': 'дональд Стерлинг, игрок прошлогодней команды НБА . жена Стерлинга подала в суд на подарки на сумму 2,6 миллиона долларов. стерлинг говорит, что он - бывшая компаньонка, которая проиграла конкурс. стерлинг обязал В. Стивиано выплатить 2,6 миллиона долларов в виде подарков после того, как его жена подала в суд. стерлинг также приобрел костюм пасхального кролика стоимостью 391 доллар, 299 долларов и еще 299 долларов.'}, {'text': 'Дональд Стерлинг обвинил Стивиано в том, что он нацелился на очень богатых пожилых мужчин. она утверждала, что Дональд Стерлинг использовал деньги пары, чтобы купить Стивиано ferrari, два bentley и range rover. Стивиано возразил, что нет ничего плохого в том, что Дональд Стерлинг дарит ей подарки.'}, {'text': 'судья Лос-Анджелеса обязал В. Стивиано выплатить более 2,6 миллионов долларов в виде подарков после того, как жена Стерлинга подала на нее в суд. -lrb- cnn -rrb- расистские высказывания Дональда Стерлинга в прошлом году стоили ем

In [56]:
from tqdm import tqdm

translated_summaries = []
for row in tqdm(df.iterrows()):
    request_data = list(row[1]["human_summaries"])
    r = send_yatranslate_request(request_data)

    for txt in r.json()["translations"]:
        translated_summaries.append(txt["text"])

100it [07:22,  4.42s/it]


In [64]:
from tqdm import tqdm

translated_original_texts = []
for row in tqdm(df.iterrows()):
    request_data = row[1]["text"]
    r = send_yatranslate_request([request_data])
    
    for txt in r.json()["translations"]:
        translated_original_texts.append(txt["text"])

100it [06:48,  4.09s/it]


In [66]:
translated_machine_summaries = translated_texts.copy()
translated_human_summaries = translated_summaries.copy()

In [65]:
print(len(translated_original_texts), len(translated_texts), len(translated_summaries))

100 1600 1100


In [67]:
new_data = []
for i, text in enumerate(translated_original_texts):
    start_machine_index = i * 16
    start_human_index = i * 11
    for inner_index in range(16):
        entry = {}
        machine_index = start_machine_index + inner_index
        human_index = start_human_index # + inner_index
        
        entry["text"] = text
        entry["summary"] = translated_human_summaries[human_index]
        entry["machine_summary"] = translated_machine_summaries[machine_index]

        entry["Актуальность"] = df.iloc[i]["relevance"][inner_index]
        entry["Последовательность"] = df.iloc[i]["consistency"][inner_index]
        entry["Беглость"] = df.iloc[i]["fluency"][inner_index]
        entry["Согласованность"] = df.iloc[i]["coherence"][inner_index]
        new_data.append(entry)
    

In [68]:
print(len(new_data))

1600


In [70]:
new_data = pd.DataFrame(new_data)

In [71]:
new_data.head()

,text,summary,machine_summary,Актуальность,Последовательность,Беглость,Согласованность
0,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...","Дональд Стерлинг, игрок прошлогодней команды Н...",1.666667,1.000000,1.000000,1.333333
1,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...","Дональд Стерлинг обвинил Стивиано в том, что о...",1.666667,2.333333,4.666667,3.000000
2,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",судья Лос-Анджелеса обязал В. Стивиано выплати...,2.333333,4.666667,4.333333,1.000000
3,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",Жена Дональда Стерлинга подала в суд на Стивиа...,3.333333,5.000000,4.333333,2.666667
4,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",расистские высказывания Дональда Стерлинга в п...,3.000000,5.000000,5.000000,4.666667


In [72]:
new_data.to_csv("translated_data.csv", index=False)